# Statistical physics of graphs and networks
## *Project:* study of the configuration model of random graphs
### *Master in physics of complex systems*
Joseph Touzet

### Problem 1: Generation of instances of the random graph model.